[View in Colaboratory](https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/WOS_SCI_SCP.ipynb)

# WOS+SCI+SCP
Merge the bibliographic datasets for Web of Scienca, Scielo and Scopus of the scientific articles of Universidad de Antioquia

For details see [merge.ipynb in Colaboratory](https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/merge.ipynb)

In [0]:
!pip install wosplus openpyxl xlrd > /dev/null

In [0]:
import wosplus as wp
import pandas as pd

##  Configure public links of  files in Google Drive
* If it is a Google Spreadsheet the corresponding file is downloaded as CSV
* If it is in excel or text file the file is downloaded  directly

To define your  own labeled IDs for public google drive files edit the next cell:

In [5]:
%%writefile drive.cfg
[FILES]
UDEA_WOS.xlsx       = 1px2IcrjCrkyu7t78Q7PAE5nzV_yuPt9t
UDEA_SCI.xlsx       = 1pWMY5P72j0Ca6D-cm7dn7Q4TBGTs4PWV
UDEA_SCP.xlsx       = 1ulCsFHzDiTmuL9TH8F58ulh0u8Z2ylKh

Overwriting drive.cfg


##  Load data bases

In [7]:
UDEA=wp.wosplus('drive.cfg')
tmp=UDEA.load_biblio('UDEA_WOS.xlsx') # creates WOS.WOS
tmp=UDEA.load_biblio('UDEA_SCI.xlsx',prefix='SCI') # creates UDEA.SCI
tmp=UDEA.load_biblio('UDEA_SCP.xlsx',prefix='SCP') # creates UDEA.SCP

/usr/local/lib/python3.6/dist-packages/wosplus/_wos_scp.py:54: FutureWarning: Using 'rename_axis' to alter labels is deprecated. Use '.rename' instead
  return df.rename_axis( dict( (key,prefix+'_'+key) for key in df.columns.values) , axis=1)


In [8]:
print('before merge: {}'.format( UDEA.WOS.shape[0]+UDEA.SCI.shape[0]+UDEA.SCP.shape[0] )  )


before merge: 24676


### WOS+SCI

In [9]:
UDEA.merge(left='WOS',right='SCI')

................................................................................

### WOS+SCI+SCP

In [10]:
UDEA.merge(left='WOS_SCI',right='SCP')

................................................................................................................................................................................................................

In [12]:
print( 'after merge: {}'.format(UDEA.WOS_SCI_SCP.shape[0]))

after merge: 15504


In [0]:
UDEA.WOS_SCI_SCP.to_excel('UDEA_WOS_SCI_SCP.xlsx',index=False)

See the resulting Excel file by opening the side bar → Files → REFRESH and right click for Download upon the generated  "'UDEA_WOS_SCI_SCP.xlsx'" fille, or just check the corresponsing DataFrame:

In [0]:
UDEA.WOS_SCI_SCP.sample()